## Cleaning & Setting Up SQL Tables

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as db

In [2]:
races = pd.read_csv('./race_stats.csv')

In [3]:
drivers = pd.read_csv('./races.csv')

In [4]:
races.tail()

,Time of race,Average speed,Pole speed,Cautions,Margin of victory,Attendance,Lead changes,date,Pole time
98,5:33:56,89.84 mph,98.9 mph,NaN,209.43 sec,"83,000",7,05/31/1915,NaN
99,6:03:46,82.474 mph,NaN,NaN,398.5 sec,"125,000",10,05/30/1914,NaN
100,6:35:05,75.933 mph,NaN,NaN,788.4 sec,NaN,8,05/30/1913,NaN
101,6:21:06,78.719 mph,NaN,NaN,623 sec,NaN,3,05/30/1912,NaN
102,6:42:08,74.59 mph,NaN,NaN,103 sec,NaN,13,05/30/1911,NaN


In [5]:
new_col = [string.lower().replace(' ', '_') for string in races.columns]

names = dict(zip(races.columns, new_col))

races = races.rename(columns = names)

In [6]:
races = races[['time_of_race', 'average_speed', 'margin_of_victory', 'lead_changes', 'date']]

races['year'] = [int(date[-4:]) for date in races['date']]

years = list(reversed(list(races['year'])))

race_dict = {y:x for (x,y) in enumerate(years, 1)}

In [7]:
races = races.dropna()

In [8]:
races['id'] = races['year'].map(race_dict)

In [9]:
drivers['race_id'] = drivers['year'].map(race_dict)

In [10]:
drivers.tail()

,finish,start,car_number,driver,car_name/entrant,make_model,status,laps,LED,winnings,year,race_id
3325,29,28,26,Zach Veach,Gainbridge,Dallara/Honda,Contact,176,0,"$334,129",2019,103
3326,30,7,18,Sebastien Bourdais,SealMaster,Dallara/Honda,Contact,176,0,"$342,129",2019,103
3327,31,33,32,Kyle Kaiser,Juncos Racing,Dallara/Chevrolet,Contact,71,0,"$205,305",2019,103
3328,32,27,81,Ben Hanley,10 Star DragonSpeed,Dallara/Honda,Mechanical,54,0,"$200,805",2019,103
3329,33,5,88,Colton Herta,Harding Steinbrenner Racing,Dallara/Honda,Mechanical,3,0,"$351,129",2019,103


In [11]:
def to_num(number):
    new = int(number.replace('$', '').replace(',', ''))
    return new

In [12]:
drivers['winnings'] = drivers['winnings'].apply(to_num)

In [13]:
drivers.describe()

,finish,start,car_number,laps,LED,winnings,year,race_id
count,3330.000000,3330.000000,3330.000000,3330.000000,3330.000000,3.330000e+03,3330.000000,3330.000000
mean,16.841441,16.821622,31.356757,134.414114,6.057958,1.127569e+05,1967.867568,53.032733
std,9.603248,9.604746,26.548901,69.400194,21.777830,2.285183e+05,30.810132,29.258399
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1911.000000,1.000000
25%,9.000000,9.000000,10.000000,71.000000,0.000000,2.335250e+03,1940.000000,28.000000
50%,17.000000,17.000000,23.000000,163.000000,0.000000,1.743700e+04,1969.000000,53.000000
75%,25.000000,25.000000,45.000000,200.000000,0.000000,1.777405e+05,1994.000000,78.000000
max,42.000000,42.000000,99.000000,200.000000,198.000000,3.048005e+06,2019.000000,103.000000


#### SQL Connection

In [14]:
from creds import uri

In [15]:
engine = db.create_engine(uri)
connection = engine.connect()
metadata = db.MetaData()

#### Table 1: Races

In [16]:
#Help with this section from LA Flex, Dan Kim example
columns_to_add = ''

for name, datatype in zip(races.columns, list(races.dtypes.values)): 
    if datatype == int:
        columns_to_add += f'{name} INT, '
    elif datatype == object:
        columns_to_add += f'{name} TEXT, '
    elif datatype == float:
        columns_to_add += f'{name} FLOAT, '

columns_to_add = "(" + columns_to_add.strip(', ') + ")"
columns_to_add

'(time_of_race TEXT, average_speed TEXT, margin_of_victory TEXT, lead_changes INT, date TEXT, year INT, id INT)'

In [17]:
query = f'CREATE TABLE races {columns_to_add}'
#connection.execute(query) 

In [18]:
values_to_add = []

for row in range(len(races)):
    values_to_add.append(tuple(races.loc[row].values))

KeyError: 41

In [19]:
for values in values_to_add:
    query = f'INSERT INTO races VALUES {str(values)}'
    connection.execute(query)

#### Table 2: Drivers

In [26]:
drivers = drivers.rename(columns = {'car_name/entrant' : 'car_name'})

In [27]:
#Help with this section from LA Flex, Dan Kim example
columns_to_add = ''

for name, datatype in zip(drivers.columns, list(drivers.dtypes.values)): 
    if datatype == int:
        columns_to_add += f'{name} INT, '
    elif datatype == object:
        columns_to_add += f'{name} TEXT, '
    elif datatype == float:
        columns_to_add += f'{name} FLOAT, '

columns_to_add = "(" + columns_to_add.strip(', ') + ")"
columns_to_add

'(finish INT, start INT, car_number INT, driver TEXT, car_name TEXT, make_model TEXT, status TEXT, laps INT, LED INT, winnings INT, year INT, race_id INT)'

In [28]:
query = f'CREATE TABLE drivers {columns_to_add}'
#connection.execute(query) 

In [31]:
def add_esc(name):
    return name.replace('\'', '_')

In [32]:
add_esc("Eddie O'Donnell")

'Eddie O_Donnell'

In [34]:
drivers['driver'] = drivers['driver'].apply(add_esc)
drivers['car_name'] = drivers['car_name'].apply(add_esc)

In [36]:
values_to_add = []

for row in range(len(drivers)):
    values_to_add.append(tuple(drivers.loc[row].values))

In [37]:
for values in values_to_add:
    query = f'INSERT INTO drivers VALUES {str(values)}'
    connection.execute(query)

In [38]:
len(drivers) + len(races)

3430